In [1]:
import pandas as pd
import csv
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sc_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
listings = pd.read_csv(listings_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

In [4]:
listings.columns.values

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access',
       'interaction', 'house_rules', 'thumbnail_url', 'medium_url',
       'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bath

In [5]:
df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost",
               "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms", 
               "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
               "reviews_per_month"]]

In [6]:
df.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,100%,NaN,f,15,95051,Apartment,Entire home/apt,1,1.0,NaN,1.0,$85.00,4,85.0,strict_14_with_grace_period,0.08
1,100%,NaN,f,15,95051,Apartment,Entire home/apt,3,1.0,1.0,1.0,$95.00,10,94.0,strict_14_with_grace_period,0.13
2,80%,NaN,f,2,94041,House,Private room,2,1.0,1.0,1.0,$80.00,0,NaN,strict_14_with_grace_period,NaN
3,100%,NaN,t,9,94301,House,Private room,1,2.0,1.0,1.0,$99.00,179,96.0,moderate,1.79
4,100%,NaN,f,1,94043,House,Private room,2,1.0,1.0,1.0,$299.00,3,100.0,moderate,0.03


In [7]:
# % of NaN values
(len(df)-df.count())/len(df)*100

host_response_rate        7.745236
host_acceptance_rate    100.000000
host_is_superhost         0.000000
host_listings_count       0.000000
zipcode                   3.087509
property_type             0.000000
room_type                 0.000000
accommodates              0.000000
bathrooms                 0.088215
bedrooms                  0.035286
beds                      0.194072
price                     0.000000
number_of_reviews         0.000000
review_scores_rating     18.966126
cancellation_policy       0.000000
reviews_per_month        18.207481
dtype: float64

In [8]:
df = df.drop(['host_acceptance_rate'], axis=1)

In [9]:
# drop NaN rows
df2=df.dropna(axis=0)

In [10]:
# % of rows droppped
(len(df)-len(df2))/len(df)*100

23.341566690190543

In [11]:
df2.shape

(4345, 15)

In [12]:
pd.options.mode.chained_assignment = None  # default='warn'

df2['host_response_rate'] = df2['host_response_rate'].astype(str)
df2['price'] = df2['price'].astype(str)

In [13]:
# clean data
pd.options.mode.chained_assignment = None  # default='warn'

df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

In [14]:
df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
del df2['host_is_superhost']

In [15]:
# select non-numeric variables and create dummies
non_num_vars = df2.select_dtypes(include=['object']).columns
df2[non_num_vars].head()

,zipcode,property_type,room_type,cancellation_policy
1,95051,Apartment,Entire home/apt,strict_14_with_grace_period
3,94301,House,Private room,moderate
4,94043,House,Private room,moderate
5,94041,House,Private room,flexible
6,95117,Townhouse,Private room,flexible


In [16]:
dummy_vars = pd.get_dummies(df2[non_num_vars])

In [17]:
dummy_vars.head()

,zipcode_94022,zipcode_94024,zipcode_94025,zipcode_94028,zipcode_94030,zipcode_94040,zipcode_94041,zipcode_94043,zipcode_94085,zipcode_94086,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [18]:
# drop non-numeric variables from df2 and add the dummies
df3=df2.drop(non_num_vars,axis=1)
df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)
df3.head()

,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
1,100.0,15,3,1.0,1.0,1.0,95.0,10,94.0,0.13,...,0,1,0,0,0,0,0,1,0,0
3,100.0,9,1,2.0,1.0,1.0,99.0,179,96.0,1.79,...,0,0,1,0,0,1,0,0,0,0
4,100.0,1,2,1.0,1.0,1.0,299.0,3,100.0,0.03,...,0,0,1,0,0,1,0,0,0,0
5,100.0,1,2,1.0,1.0,1.0,99.0,61,91.0,1.00,...,0,0,1,0,1,0,0,0,0,0
6,100.0,1,3,1.0,0.0,2.0,61.0,6,93.0,3.60,...,0,0,1,0,1,0,0,0,0,0


In [19]:
df4 = pd.get_dummies(df2, prefix=non_num_vars, columns=non_num_vars)
df4.shape

(4345, 114)

In [20]:
# split into test and training data
np.random.seed(1)
indices = np.random.permutation(len(df4))
train_size = int(round(0.8*len(df4)))
test_size = len(df4)-train_size

y = df4['price']
x = df4.drop('price', axis =1)

x.train = x.iloc[indices[0:train_size]]
y.train = y.iloc[indices[0:train_size]]
x.test = x.iloc[indices[train_size+1:]]
y.test = y.iloc[indices[train_size+1:]]

x2 = x.train.as_matrix()
y2 = y.train.as_matrix()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [21]:
import statsmodels.api as sm
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()
print(olsres.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     31.43
Date:                Sat, 15 Sep 2018   Prob (F-statistic):               0.00
Time:                        21:58:50   Log-Likelihood:                -21749.
No. Observations:                3476   AIC:                         4.371e+04
Df Residuals:                    3371   BIC:                         4.435e+04
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1699      0.247      0.688      0.4

In [22]:
# different method
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=1)

In [23]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 2870.698, test: 6131.234
R^2 train: 0.918, test: 0.733
